In [14]:
import param as pm
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv

In [15]:
class TokenEngineering(pm.Parameterized):
    monthly_salary = pm.Integer(5000, bounds=(1500,5000), step=500)
    number_employees = pm.Integer(7, bounds=(3, 12), step=1)
    number_months = pm.Integer(24, bounds=(3, 24), step=1)
    monthly_contract_size = pm.Integer(7500, bounds=(6000, 10000), step=50)
    number_of_initial_contracts = pm.Integer(2, bounds=(1, 10), step=1)
    new_contracts_per_month = pm.Number(0.33, bounds=(0, 3), step=0.33)
    office_expense = pm.Integer(7000, bounds=(3000,7000), step=50)
    
    def salary_costs(self):
        cummulative = [i*self.number_employees*self.monthly_salary for i in range(self.number_months)]
        return cummulative
    
    def office_expenses(self):
        cummulative = [i*self.office_expense for i in range(self.number_months)]
        return cummulative
    
    def costs(self):
        return [a+b for a,b in zip(self.salary_costs(), self.office_expenses())]
    
    def number_of_contracts(self):
        cummulative = [i*self.new_contracts_per_month + self.number_of_initial_contracts for i in range(self.number_months)]
        return cummulative
    
    def contract_revenue(self):
        number_of_contracts = self.number_of_contracts()
        cummulative = [i*self.monthly_contract_size*number_of_contracts[i] for i in range(self.number_months)]
        return cummulative
    
    def ltf_treasury(self):
        return [a-b for a,b in zip(self.contract_revenue(),self.costs())]
    
    def cummulative_data(self):
        data = pd.DataFrame({
            'Contract Revenue': self.contract_revenue(), 
            'Number of Contracts': self.number_of_contracts(),
            'Salary Costs': self.salary_costs(), 
            'Net Profit': self.ltf_treasury(),
            'Office Expenses': self.office_expenses()})
        data.index.name = 'Month'
        return data
    
    def results(self):
        return self.cummulative_data().iloc[[-1]]
    
    def results_view(self):
        return self.results().reset_index().hvplot.table(title="Results")
    
    def chart_view(self):
        return self.cummulative_data().hvplot.line(title='Cumulative Revenue, Costs, and Profit') * hv.HLine(0).opts(color='black', line_width=1.2)
    
    def view_te(self):
        return lambda te: pn.Row(te, pn.Column(te.chart_view, te.results_view))

In [16]:
t = TokenEngineering()
t.view_te()(t)

Row
    [0] Column(margin=5, name='TokenEngineering', width=300)
        [0] StaticText(value='<b>TokenEngineering</b>')
        [1] IntSlider(end=5000, name='Monthly salary', start=1500, step=500, value=5000)
        [2] IntSlider(end=12, name='Number employees', start=3, value=7)
        [3] IntSlider(end=24, name='Number months', start=3, value=24)
        [4] IntSlider(end=10000, name='Monthly contract size', start=6000, step=50, value=7500)
        [5] IntSlider(end=10, name='Number of initial c..., start=1, value=2)
        [6] FloatSlider(end=3, name='New contracts per month', step=0.33, value=0.33)
        [7] IntSlider(end=7000, name='Office expense', start=3000, step=50, value=7000)
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)